## 준비하기
1. 아래 예제를 실행하기 위해서는 사전에 ./setup.sh 파일을 실행하거나 ./start_notebook.sh 로 노트북을 구동하여 의존성 라이브러리들을 미리 설치해두셔야 합니다.
2. qdrant서버가 떠 있어야 정상 동작합니다. ./start_qdrant.sh 를 실행하여 qdrant 서버를 띄워주세요.

## 목적:
- React (Reasoning + Acting)를 LangGraph 데이터 흐름에 적용
- Qdrant를 활용한 RAG 검색 시스템 구현
- RAGAS를 사용하여 응답 품질 평가 수행

## 왜 필요한가?
- LLM이 생성한 응답이 신뢰할 수 있는지 평가할 수 있음
- 검색된 문서가 실제로 도움이 되는지 검증 가능
- RAG 기반 AI 시스템의 성능을 향상시킬 수 있음

## 주요 개념:
- **React (Reasoning and Acting)**: AI가 자체적으로 검색하고 답변을 생성하는 방식
- **RAG (Retrieval-Augmented Generation)**: 벡터 검색을 활용한 LLM 응답 생성 기법
- **Qdrant**: 고속 벡터 검색을 지원하는 데이터베이스
- **RAGAS**: RAG 응답의 신뢰성과 품질을 평가하는 프레임워크

In [1]:
# 라이브러리 로딩 (아래 부분에서 오류가 발생하면 pip -r ./requirements.txt 로 의존성을 설치해주세요)
import os
from langgraph.graph import StateGraph, START
from typing import TypedDict, Annotated
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from ragas.metrics import answer_relevancy, faithfulness, context_precision, context_recall
from ragas import evaluate

In [2]:
# Qdrant 클라이언트 설정
qdrant_client = QdrantClient(host="localhost", port=6333)
collection_name = "react_ragas_vectors"

# Qdrant 컬렉션 생성 (없을 경우 자동 생성)
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=384, distance=Distance.COSINE)  # 384는 임베딩 차원 수
    )

In [3]:
# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [4]:
# OpenAI API 키 설정
openai_api_key = os.environ.get("OPENAI_API_KEY", "your_api_key")
if openai_api_key == "your_api_key":
    print("WARNING: OPENAI_API_KEY가 환경 변수에 설정되지 않았습니다. 기본값을 사용합니다.")

# Qdrant 기반 벡터 저장소 생성
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings
)

# LangGraph에서 사용할 상태 타입 정의
class QueryState(TypedDict):
    text: str
    vector: list[float] | None
    response: str | None

# Reasoning 단계: 텍스트를 벡터로 변환
def generate_vector(state: QueryState) -> Annotated[QueryState, "store_in_qdrant", "search_in_qdrant"]:
    vector = embeddings.embed_query(state["text"])  # 최신 메서드 사용
    print(f"'{state['text']}' → 벡터 변환 완료")
    return {"text": state["text"], "vector": vector, "response": None}

# Acting 단계: Qdrant에 벡터 저장
def store_in_qdrant(state: QueryState) -> QueryState:
    vectorstore.add_texts([state["text"]])
    print(f"'{state['text']}' 벡터가 Qdrant에 저장됨")
    return state

# Reasoning 단계: Qdrant에서 유사한 벡터 검색 및 LLM 응답 생성
def search_in_qdrant(state: QueryState) -> QueryState:
    retriever = vectorstore.as_retriever()
    
    # LangChain을 활용한 Retrieval-Augmented Generation (RAG)
    llm = OpenAI(api_key=openai_api_key)
    qa_chain = RetrievalQA(llm=llm, retriever=retriever)
    
    response = qa_chain.invoke({"query": state["text"]})  # 최신 방식 적용
    print(f"React 기반 LLM 응답: {response}")
    
    return {"text": state["text"], "vector": state["vector"], "response": response}

# RAGAS 기반 응답 평가
def evaluate_ragas(state: QueryState) -> None:
    # 평가 메트릭 설정
    scores = evaluate(
        metrics=[answer_relevancy, faithfulness, context_precision, context_recall],
        query=state["text"],
        response=state["response"]
    )
    print(f"RAGAS 평가 결과: {scores}")

In [5]:
# StateGraph 정의
graph = StateGraph(state_schema=QueryState)

# 노드 추가
graph.add_node("generate_vector", generate_vector)  # 텍스트 → 벡터 변환 (Reasoning)
graph.add_node("store_in_qdrant", store_in_qdrant)  # 벡터 저장 (Acting)
graph.add_node("search_in_qdrant", search_in_qdrant)  # 벡터 검색 및 LLM 응답 생성 (Reasoning)
graph.add_node("evaluate_ragas", evaluate_ragas)  # RAGAS 기반 평가 수행

# 데이터 흐름 정의
graph.add_edge(START, "generate_vector")
graph.add_edge("search_in_qdrant", "evaluate_ragas")

In [6]:
# StateGraph 변환 및 실행
app = graph.compile()
app.invoke({"text": "What is LangChain and how does it work?"})

'What is LangChain and how does it work?' → 벡터 변환 완료


{'text': 'What is LangChain and how does it work?',
 'vector': [-0.06600774079561234,
  -0.01962524652481079,
  0.02879750169813633,
  -0.010752083733677864,
  -0.0441880039870739,
  0.038088079541921616,
  0.04818052798509598,
  0.04874632880091667,
  0.09124452620744705,
  -0.027292635291814804,
  0.06509104371070862,
  0.0068229506723582745,
  -0.022724106907844543,
  0.00995591003447771,
  -0.07512889802455902,
  -0.0069334181025624275,
  -0.074467234313488,
  0.07518936693668365,
  -0.014002789743244648,
  -0.11970202624797821,
  0.01220423262566328,
  -0.00799890048801899,
  -0.00038113645859993994,
  0.01777704805135727,
  0.03613726794719696,
  -0.06344343721866608,
  0.010198208503425121,
  -0.011951502412557602,
  0.06419592350721359,
  0.0019424797501415014,
  0.027413668110966682,
  0.07791462540626526,
  -0.0024773962795734406,
  0.014552310109138489,
  -0.14665436744689941,
  0.12705600261688232,
  -0.01976754330098629,
  -0.021391242742538452,
  0.006127981003373861,
  0